# **AAHPS: ASSIGNMENT 4**

**Author**: Nina Mislej
<br/>**Student number**: 63200016

### **Exercise Description**
We are implementing 3 different basic search algorithms: **grid search**, **random search** and **first descent local optimization**.
<br/>The assignment consists of **finding values** at specific points in **2D space** of three functions: **Rosenbrock**, **Ackley** and **Rastrigin**. 
<br/>This assignment serves as an introduction to **continuous optimization** using functions available in ***smoof*** package in **R**. 


In [61]:
# setting up the package and enviroment
import numpy as np
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

numpy2ri.activate() # automatic conversion from numpy to R arrays
smoof = importr("smoof") # importing R smoof package
GenSA = importr("GenSA") # importing R GenSA package 

f_rosenbrock = smoof.makeRosenbrockFunction(2)
f_ackley = smoof.makeAckleyFunction(2)
f_rastrigin = smoof.makeRastriginFunction(2)

# testing the function:
print(f"the value of the ROSENBROCK function at point (1, 1): {f_rosenbrock([1, 1])[0]}") 
print(f"the value of the ACKLEY function at point (1, 1): {f_ackley([1, 1])[0]}") 
print(f"the value of the RASTRIGIN function at point (1, 1): {f_rastrigin([1, 1])[0]}") 



the value of the ROSENBROCK function at point (1, 1): 0.0
the value of the ACKLEY function at point (1, 1): 3.6253849384403627
the value of the RASTRIGIN function at point (1, 1): 2.0


Setting the **bounds** of all three functions:

In [62]:
constr_rosenbrock = [smoof.getLowerBoxConstraints(f_rosenbrock), smoof.getUpperBoxConstraints(f_rosenbrock)]
constr_ackley = [smoof.getLowerBoxConstraints(f_ackley), smoof.getUpperBoxConstraints(f_ackley)]
constr_rastrigin = [smoof.getLowerBoxConstraints(f_rastrigin), smoof.getUpperBoxConstraints(f_rastrigin)]

print(f"Bounds for ROSENBROCK function: {constr_rosenbrock}")
print(f"Bounds for ACKLEY function: {constr_ackley}")
print(f"Bounds for RASTRIGIN function: {constr_rastrigin}")

Bounds for ROSENBROCK function: [array([-5., -5.]), array([10., 10.])]
Bounds for ACKLEY function: [array([-32.768, -32.768]), array([32.768, 32.768])]
Bounds for RASTRIGIN function: [array([-5.12, -5.12]), array([5.12, 5.12])]


Let's look at the **optimum**, just so we can make better comparisons when testing our algorithms.

In [63]:
out = GenSA.GenSA(fn= f_rosenbrock, lower=constr_rosenbrock[0], upper=constr_rosenbrock[1])
value = out.rx2("value")
par = out.rx2("par")
print(f"ROSENBROCK:\nvalue: {value}    par: {par}\n") 

#---------------------------------------------------------------------------------------------#

out = GenSA.GenSA(fn= f_ackley, lower=constr_ackley[0], upper=constr_ackley[1])
value = out.rx2("value")
par = out.rx2("par")
print(f"ACKLEY:\nvalue: {value}    par: {par}\n") 

#---------------------------------------------------------------------------------------------#

out = GenSA.GenSA(fn= f_rastrigin, lower=constr_rastrigin[0], upper=constr_rastrigin[1])
value = out.rx2("value")
par = out.rx2("par")
print(f"RASTRIGIN:\nvalue: {value}    par: {par}\n")

ROSENBROCK:
value: [3.75914148e-20]    par: [1. 1.]

ACKLEY:
value: [4.4408921e-16]    par: [ 5.16196753e-17 -2.27024392e-16]

RASTRIGIN:
value: [0.]    par: [4.67346329e-10 6.07685317e-11]



We could also find **maximum** in **R** but using **rpy2** it causes too many complications so i will just show the **output** rounded to 3 decimals:




| MIN                 | x1      | x2      | value   |
|---------------------|---------|---------|---------|
| Rosenbrock          | 1       | 1       | 0       |
| Ackley              | 0       | 0       | 0       |
| Rastrigin           | 0       | 0       | 0       |

<br/>

| MAX                 | x1      | x2      | value   |
|---------------------|---------|---------|---------|
| Rosenbrock          | 10      |-5       |1102581  |
| Ackley              |-32.5    |-32.5    |22.32    |
| Rastrigin           | 4.523   | 4.523   |80.707   |


## **GRID SEARCH**

We are implementing a grid search algorithm to evaluate the 3 given functions on a discrete grid of points within the specified bounds for each function. The size of the grid is 1. 
<br/>**NOTE** : The point (**0**, **0**) should always be included in the search. This defines our grid perfectly, because we can take **integers** as bounds. 

In [64]:
def grid_search(lower_bound, upper_bound, function):
    min = [float('inf'), 0, 0]
    max = [float('-inf'), 0, 0]
    iterations = 0

    for x in range(lower_bound, upper_bound):
        for y in range(lower_bound, upper_bound):
            curr = function([x,y])[0]
            curr = round(curr, ndigits=3)
            iterations = iterations + 1
            if curr < min[0]: min = [curr, x, y]
            if curr > max[0]: max = [curr, x, y]
    
    return min, max, iterations

In [65]:
lower_bound =  int(np.ceil(constr_rosenbrock[0][0]))
upper_bound =  int(np.floor(constr_rosenbrock[1][0])) + 1
min, max, iterations = grid_search(lower_bound, upper_bound, f_rosenbrock)

print(f"ROSENBROCK:        lower bound: {lower_bound}          upper_bound: {upper_bound - 1}")
print(f"ROSENBROCK:        minimum: {min[0]}             coordinates: {min[1:3]}")
print(f"ROSENBROCK:        maximum: {max[0]}       coordinates: {max[1:3]}")
print(f"ROSENBROCK:        points tested: {iterations}\n")

#--------------------------------------------------------------------------------------------------#

lower_bound =  int(np.ceil(constr_ackley[0][0]))
upper_bound =  int(np.floor(constr_ackley[1][0])) + 1
min, max, iterations = grid_search(lower_bound, upper_bound, f_ackley)

print(f"ACKLEY:            lower bound: {lower_bound}         upper_bound: {upper_bound - 1}")
print(f"ACKLEY:            minimum: {min[0]}             coordinates: {min[1:3]}")
print(f"ACKLEY:            maximum: {max[0]}          coordinates: {max[1:3]}")
print(f"ACKLEY:            points tested: {iterations}\n")

#-------------------------------------------------------------------------------------------------#

lower_bound =  int(np.ceil(constr_rastrigin[0][0]))
upper_bound =  int(np.floor(constr_rastrigin[1][0])) + 1
min, max, iterations = grid_search(lower_bound, upper_bound, f_rastrigin)

print(f"RASTRIGIN:         lower bound: {lower_bound}          upper_bound: {upper_bound - 1}")
print(f"RASTRIGIN:         minimum: {min[0]}             coordinates: {min[1:3]}")
print(f"RASTRIGIN:         maximum: {max[0]}            coordinates: {max[1:3]}")
print(f"RASTRIGIN:         points tested: {iterations}\n")

ROSENBROCK:        lower bound: -5          upper_bound: 10
ROSENBROCK:        minimum: 0.0             coordinates: [1, 1]
ROSENBROCK:        maximum: 1102581.0       coordinates: [10, -5]
ROSENBROCK:        points tested: 256

ACKLEY:            lower bound: -32         upper_bound: 32
ACKLEY:            minimum: 0.0             coordinates: [0, 0]
ACKLEY:            maximum: 19.967          coordinates: [-32, -32]
ACKLEY:            points tested: 4225

RASTRIGIN:         lower bound: -5          upper_bound: 5
RASTRIGIN:         minimum: 0.0             coordinates: [0, 0]
RASTRIGIN:         maximum: 50.0            coordinates: [-5, -5]
RASTRIGIN:         points tested: 121



We notice that the problem of grid search given our **discrete grid** is that our size 1 is pretty large. The minimum and maximum for the **first function** are **precise** because the pair of optimal coordinates are integers. For the second one the result is not that bad, but looking at the maximum for the **last function** the result is **very off**. Another thing we can see is that the **second function** has to go through quite **a lot of points**. 

Let's see if the results using **random search** would be better.

## **RANDOM SEARCH**

We will implement a random search function that searches the 2D space uniformly randomly within the specified bounds for each function.
<br/>**NOTE**: we will make **1000** calls


In [66]:
def random_search(lower_bound, upper_bound, function, limit):
    min = [float('inf'), 0, 0]
    max = [float('-inf'), 0, 0]
    sum = 0

    points = np.random.uniform(lower_bound, upper_bound, (limit,2))

    for point in points:
        curr = function(point)[0]
        sum = sum + curr   
        if curr < min[0]: min = [round(curr, ndigits=3), round(point[0], ndigits=3), round(point[1], ndigits=3)]
        if curr > max[0]: max = [round(curr, ndigits=3), round(point[0], ndigits=3), round(point[1], ndigits=3)]
    
    mean = round(sum / limit, ndigits=3)

    return min, max, mean


In [67]:
limit = 1000

lower_bound =  constr_rosenbrock[0][0]
upper_bound =  constr_rosenbrock[1][0]
min, max, mean = random_search(lower_bound, upper_bound, f_rosenbrock, limit)

print(f"ROSENBROCK:        lower bound: {lower_bound}          upper_bound: {upper_bound}")
print(f"ROSENBROCK:        minimum: {min[0]}             coordinates: {min[1:3]}")
print(f"ROSENBROCK:        maximum: {max[0]}       coordinates: {max[1:3]}")
print(f"ROSENBROCK:        mean: {mean}\n")

#--------------------------------------------------------------------------------------------------#

lower_bound =  constr_ackley[0][0]
upper_bound =  constr_ackley[1][0]
min, max, mean = random_search(lower_bound, upper_bound, f_ackley, limit)

print(f"ACKLEY:            lower bound: {lower_bound}      upper_bound: {upper_bound}")
print(f"ACKLEY:            minimum: {min[0]}             coordinates: {min[1:3]}")
print(f"ACKLEY:            maximum: {max[0]}              coordinates: {max[1:3]}")
print(f"ACKLEY:            mean: {mean}\n")

#-------------------------------------------------------------------------------------------------#

lower_bound =  constr_rastrigin[0][0]
upper_bound =  constr_rastrigin[1][0]
min, max, mean = random_search(lower_bound, upper_bound, f_rastrigin, limit)

print(f"RASTRIGIN:         lower bound: {lower_bound}         upper_bound: {upper_bound}")
print(f"RASTRIGIN:         minimum: {min[0]}             coordinates: {min[1:3]}")
print(f"RASTRIGIN:         maximum: {max[0]}            coordinates: {max[1:3]}")
print(f"RASTRIGIN:         mean: {mean}\n")

ROSENBROCK:        lower bound: -5.0          upper_bound: 10.0
ROSENBROCK:        minimum: 0.036             coordinates: [1.088, 1.2]
ROSENBROCK:        maximum: 1075504.624       coordinates: [9.982, -4.056]
ROSENBROCK:        mean: 118526.462

ACKLEY:            lower bound: -32.768      upper_bound: 32.768
ACKLEY:            minimum: 4.481             coordinates: [-0.686, 0.846]
ACKLEY:            maximum: 22.275              coordinates: [27.518, 28.508]
ACKLEY:            mean: 20.181

RASTRIGIN:         lower bound: -5.12         upper_bound: 5.12
RASTRIGIN:         minimum: 0.898             coordinates: [-0.062, -0.028]
RASTRIGIN:         maximum: 79.144            coordinates: [4.515, 4.434]
RASTRIGIN:         mean: 37.17



Here the results are more diverse, the **minimum** is not very precise and neither is the **maximum**, but the second and the third function have better results for maximal value.

## **LOCAL SEARCH**

We will implement a random search function that searches the 2D space uniformly randomly within the specified bounds for each function.
<br/>**NOTE**: we will make **1000** calls

In [68]:
def local_search(lower_bound, upper_bound, function, limit, sum):
    iterations = 0
    calls = 1

    point = np.random.uniform(lower_bound, upper_bound, 2)
    curr = function(point)[0] 
    sum = sum + curr

    while(iterations <= limit):
        for i in range(1,101): 
            neighbour = [point[0] + np.random.uniform(-0.1, 0.1), point[1] + np.random.uniform(-0.1, 0.1)]
            
            if neighbour[0] < lower_bound or neighbour[0] > upper_bound: continue
            if neighbour[1] < lower_bound or neighbour[1] > upper_bound: continue

            next = function(neighbour)[0] 
            sum = sum + next
            if next < curr:
                point = neighbour
                curr = next
                calls = calls + i
                break

            if i == 100: 
                calls = calls + i
                return [round(curr, ndigits=3), round(point[0], ndigits=3), round(point[1], ndigits=3)], iterations, calls, sum
            
        iterations = iterations + 1
    return [round(curr, ndigits=3), round(point[0], ndigits=3), round(point[1], ndigits=3)], iterations, calls, sum

In [69]:
limit = 1000

sum = 0
overall_calls = 0
lower_bound =  constr_rosenbrock[0][0]
upper_bound =  constr_rosenbrock[1][0]
min_overall = [float('inf'), 0, 0]

for i in range(1,11):
    min, iterations, calls, sum = local_search(lower_bound, upper_bound, f_rosenbrock, limit, sum)

    print(f"ROSENBROCK ({i}):        lower bound: {lower_bound}          upper_bound: {upper_bound}")
    print(f"ROSENBROCK ({i}):        minimum: {min[0]}             coordinates: {min[1:3]}")
    print(f"ROSENBROCK ({i}):        iterations: {iterations}              calls: {calls}\n")

    if min[0] < min_overall[0]: min_overall = [min[0], min[1], min[2]]
    overall_calls = overall_calls + calls

print(f"\nROSENBROCK (overall):        lower bound: {lower_bound}          upper_bound: {upper_bound}")
print(f"ROSENBROCK (overall):        minimum: {min_overall[0]}             coordinates: {min_overall[1:3]}")
print(f"ROSENBROCK (overall):        mean: {round(sum / overall_calls, ndigits=3)}")

#-----------------------------------------------------------------------------------------------------------#
print("\n--------------------------------------------------------------------------------------------\n")

sum = 0
overall_calls = 0
lower_bound =  constr_ackley[0][0]
upper_bound =  constr_ackley[1][0]
min_overall = [float('inf'), 0, 0]

for i in range(1,11):
    min, iterations, calls, sum = local_search(lower_bound, upper_bound, f_ackley, limit, sum)

    print(f"ACKLEY ({i}):        lower bound: {lower_bound}          upper_bound: {upper_bound}")
    print(f"ACKLEY ({i}):        minimum: {min[0]}             coordinates: {min[1:3]}")
    print(f"ACKLEY ({i}):        iterations: {iterations}              calls: {calls}\n")

    if min[0] < min_overall[0]: min_overall = [min[0], min[1], min[2]]
    overall_calls = overall_calls + calls

print(f"\nACKLEY (overall):        lower bound: {lower_bound}          upper_bound: {upper_bound}")
print(f"ACKLEY (overall):        minimum: {min_overall[0]}             coordinates: {min_overall[1:3]}")
print(f"ACKLEY (overall):        mean: {round(sum / overall_calls, ndigits=3)}")

#-----------------------------------------------------------------------------------------------------------#
print("\n--------------------------------------------------------------------------------------------\n")

sum = 0
overall_calls = 0
lower_bound =  constr_rastrigin[0][0]
upper_bound =  constr_rastrigin[1][0]
min_overall = [float('inf'), 0, 0]

for i in range(1,11):
    min, iterations, calls, sum = local_search(lower_bound, upper_bound, f_rastrigin, limit, sum)

    print(f"RASTRIGIN ({i}):        lower bound: {lower_bound}          upper_bound: {upper_bound}")
    print(f"RASTRIGIN ({i}):        minimum: {min[0]}             coordinates: {min[1:3]}")
    print(f"RASTRIGIN ({i}):        iterations: {iterations}             calls: {calls}\n")

    if min[0] < min_overall[0]: min_overall = [min[0], min[1], min[2]]
    overall_calls = overall_calls + calls

print(f"\nRASTRIGIN (overall):        lower bound: {lower_bound}          upper_bound: {upper_bound}")
print(f"RASTRIGIN (overall):        minimum: {min_overall[0]}             coordinates: {min_overall[1:3]}")
print(f"RASTRIGIN (overall):        mean: {round(sum / overall_calls, ndigits=3)}")

    


ROSENBROCK (1):        lower bound: -5.0          upper_bound: 10.0
ROSENBROCK (1):        minimum: 1.322             coordinates: [2.149, 4.622]
ROSENBROCK (1):        iterations: 82              calls: 396

ROSENBROCK (2):        lower bound: -5.0          upper_bound: 10.0
ROSENBROCK (2):        minimum: 15.471             coordinates: [-2.933, 8.598]
ROSENBROCK (2):        iterations: 54              calls: 836

ROSENBROCK (3):        lower bound: -5.0          upper_bound: 10.0
ROSENBROCK (3):        minimum: 3.409             coordinates: [2.84, 8.052]
ROSENBROCK (3):        iterations: 50              calls: 199

ROSENBROCK (4):        lower bound: -5.0          upper_bound: 10.0
ROSENBROCK (4):        minimum: 3.957             coordinates: [2.989, 8.935]
ROSENBROCK (4):        iterations: 109              calls: 700

ROSENBROCK (5):        lower bound: -5.0          upper_bound: 10.0
ROSENBROCK (5):        minimum: 0.001             coordinates: [1.021, 1.044]
ROSENBROCK (5): 